# Liz Young - Code Review - Technical Interview Presentation

<hr>

In this notebook I will walk through code I wrote to start analyzing reviews of musical instruments from Amazon purchases.

The goal of this project is to clean, transform, and organize the data to prepare for building models to predict review scores based on reviews provided by purchasers.

I selected [this dataset](https://www.kaggle.com/datasets/eswarchandt/amazon-music-reviews?resource=download) from Kaggle, after searching for semistructured and unstructured datasets, this one resonated with me.

<hr>

I set up a private github repo tracking my progress with this project, you can find via [this link](https://github.com/completelyAbsorbed/amazon-musical-instrument-reviews), containing ...
- commits of this project from start to finish
- a progress tracker document I designed and used to execute the PACE methodology on this project
- screen captures of my git commits demonstrating competency with git, github, and commandline
- the original database sourced from Kaggle
- this Jupyter notebook
- any files produced by this notebook

<hr>

### *Thank you for your time and consideration*

<hr>

# Import

In [ ]:
# import packages and libraries
import pandas as pd

# Load Original Data

In [7]:
# define the filename where python can find the original dataset
data_filename = "Musical_instruments_reviews.csv"

# read in the original dataset to a pandas DataFrame
data_raw = pd.read_csv(data_filename)

# Initial Inspection of Data

<hr>

*Learn more about the data using methods from the pandas DataFrame type. <br> <br>
Demonstrating competency with software design pattern: Object-Oriented Programming.*

In [9]:
# look at the first 5 rows of the original dataset
print(data_raw.head())

       reviewerID        asin  \
0  A2IBPI20UZIR0U  1384719342   
1  A14VAT5EAX3D9S  1384719342   
2  A195EZSQDW3E21  1384719342   
3  A2C00NNG1ZQQG2  1384719342   
4   A94QU4C90B1AX  1384719342   

                                       reviewerName   helpful  \
0  cassandra tu "Yeah, well, that's just like, u...    [0, 0]   
1                                              Jake  [13, 14]   
2                     Rick Bennette "Rick Bennette"    [1, 1]   
3                         RustyBill "Sunday Rocker"    [0, 0]   
4                                     SEAN MASLANKA    [0, 0]   

                                          reviewText  overall  \
0  Not much to write about here, but it does exac...      5.0   
1  The product does exactly as it should and is q...      5.0   
2  The primary job of this device is to block the...      5.0   
3  Nice windscreen protects my MXL mic and preven...      5.0   
4  This pop filter is great. It looks and perform...      5.0   

                   

In [10]:
# look at the last 5 rows of the original dataset
print(data_raw.tail())

           reviewerID        asin             reviewerName helpful  \
10256  A14B2YH83ZXMPP  B00JBIVXGC          Lonnie M. Adams  [0, 0]   
10257   A1RPTVW5VEOSI  B00JBIVXGC       Michael J. Edelman  [0, 0]   
10258   AWCJ12KBO5VII  B00JBIVXGC         Michael L. Knapp  [0, 0]   
10259  A2Z7S8B5U4PAKJ  B00JBIVXGC  Rick Langdon "Scriptor"  [0, 0]   
10260  A2WA8TDCTGUADI  B00JBIVXGC          TheTerrorBeyond  [0, 0]   

                                              reviewText  overall  \
10256            Great, just as expected.  Thank to all.      5.0   
10257  I've been thinking about trying the Nanoweb st...      5.0   
10258  I have tried coated strings in the past ( incl...      4.0   
10259  Well, MADE by Elixir and DEVELOPED with Taylor...      4.0   
10260  These strings are really quite good, but I wou...      4.0   

                                                 summary  unixReviewTime  \
10256                                         Five Stars      1405814400   
10257  Long 

In [15]:
# print a summary of the size, non-null counts, and column variable types of the original dataset
print(data_raw.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10261 entries, 0 to 10260
Data columns (total 9 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   reviewerID      10261 non-null  object 
 1   asin            10261 non-null  object 
 2   reviewerName    10234 non-null  object 
 3   helpful         10261 non-null  object 
 4   reviewText      10254 non-null  object 
 5   overall         10261 non-null  float64
 6   summary         10261 non-null  object 
 7   unixReviewTime  10261 non-null  int64  
 8   reviewTime      10261 non-null  object 
dtypes: float64(1), int64(1), object(7)
memory usage: 721.6+ KB
None


The original dataset is 9 columns and 10261 rows.

The columns provided are ... 
> 'reviewerID'
    <br>    'asin'
    <br>    'reviewerName'
    <br>    'helpful'
    <br>    'reviewText'
    <br>    'overall'
    <br>    'summary'
    <br>    'unixReviewTime'
    <br>    'reviewTime'
    
The original dataset contains no Null values.

Column 'overall' is type float64, 'unixReviewTime' is type int64, the rest are type object.

The original dataset uses ~721.6 KB in memory.


In [19]:
# examine count, mean, standard deviation, minimum, maximum, and quartiles of numeric columns in the original dataset
print(data_raw.describe())

            overall  unixReviewTime
count  10261.000000    1.026100e+04
mean       4.488744    1.360606e+09
std        0.894642    3.779735e+07
min        1.000000    1.095466e+09
25%        4.000000    1.343434e+09
50%        5.000000    1.368490e+09
75%        5.000000    1.388966e+09
max        5.000000    1.405987e+09


'overall' is the reviewer's score per the [data dictionary](https://www.kaggle.com/datasets/eswarchandt/amazon-music-reviews?resource=download). It ranges from 1 to 5.

'unixReviewTime' represents unix time of the review, and is directly related to 'reviewTime' (not numeric, so not summarised here).

Next, it would be helpful to inspect each of the non-numeric columns, and ascertain their features and usefulness.

### 'reviewTime'

In [22]:
# make a subset of the original dataset consisting of the time variables for inspection
data_time = data_raw[['unixReviewTime','reviewTime']]
print(data_time.head())

   unixReviewTime   reviewTime
0      1393545600  02 28, 2014
1      1363392000  03 16, 2013
2      1377648000  08 28, 2013
3      1392336000  02 14, 2014
4      1392940800  02 21, 2014


Time that reviews are posted doesn't pertain to the content of the reviews, so it could make sense to discard both columns, though it may be convenient and useful to first segment the data into train, test, and validation splits in distinct time periods.

### 'summary'

In [23]:
# examine the 'summary' variable of the original dataset
print(data_raw['summary'].head())

0                                     good
1                                     Jake
2                     It Does The Job Well
3            GOOD WINDSCREEN FOR THE MONEY
4    No more pops when I record my vocals.
Name: summary, dtype: object


'summary' is a string variable, probably shorter than 'reviewText'. It could be informative to examine the distribution of lengths of entries in 'summary'.

In [25]:
# assign the lengths of the 'summary' column to a new variable
lengths_summary = data_raw['summary'].str.len()

# print basic distributional measures of the lengths
print(lengths_summary.describe())

count    10261.000000
mean        24.348407
std         15.599231
min          1.000000
25%         13.000000
50%         21.000000
75%         32.000000
max        128.000000
Name: summary, dtype: float64


The lengths of strings in 'summary' range from 1 to 128. 

These may or may not be correlated with the review score. 

In a NLP model, it could be valuable to tokenize the 'summary' column for training.

### 'reviewText'

In [26]:
# examine the 'reviewText' variable of the original dataset
print(data_raw['reviewText'].head())

0    Not much to write about here, but it does exac...
1    The product does exactly as it should and is q...
2    The primary job of this device is to block the...
3    Nice windscreen protects my MXL mic and preven...
4    This pop filter is great. It looks and perform...
Name: reviewText, dtype: object


'reviewText' is also a string variable. As with 'summary', it may be informative to consider the lengths of these entries.

In [27]:
# assign the lengths of the 'reviewText' column to a new variable
lengths_review = data_raw['reviewText'].str.len()

# print basic distributional measures of the lengths
print(lengths_review.describe())

count    10254.000000
mean       486.260776
std        613.514024
min          9.000000
25%        163.000000
50%        285.000000
75%        552.000000
max      11310.000000
Name: reviewText, dtype: float64


The lengths of entries in 'reviewText' range in length from 9 to 11310. 

As with 'summary' this information may be useful for a model predicting the review score, and tokenizing these entries would be a step to strongly consider for an NLP model.

It may be useful to transform the length representations of 'reviewText' and 'summary' to buckets, and/or change the scale (i.e. take a logarithm of the length).

### 'helpful'

In [28]:
# examine the 'helpful' variable of the original dataset
print(data_raw['helpful'].head())

0      [0, 0]
1    [13, 14]
2      [1, 1]
3      [0, 0]
4      [0, 0]
Name: helpful, dtype: object


The 'helpful' column contains pairs of numbers, the first representing number of people who found the review helpful, with the second representing the total number of people who responded to a prompt asking whether or not the review was helpful.

This column could be transformed into multiple new variables that could be informative for predicting the review score such as; number of responses, portion of responses asserting the review was helpful, portion of responses asserting the review was not helpful.

These new variables could also be useful targets for a machine-learning model.

It may be informative to explore the relation between these variables and the 'overall' review score.

Before moving on, write functions to decompose 'helpful' into integers. *(functional programming)*

In [71]:
# define a function to decompose 'string_pair' into integers
def decompose_string_pair(string_pair) : 
    """
    Decompose a single string into a pair of integers
    
        Parameters:
            string_pair (str): a string representing a pair of integers, i.e. "[13, 14]"
        
        Returns:
            found_helpful (int): an integer representing the number that found the review helpful
            total_helpful (int): an integer representing the total number that responded helpful or not
            
        References:
            https://stackoverflow.com/questions/17416132/extracting-integers-from-a-string-of-ordered-pairs-in-python
    """
    # split the string and remove brackets and comma via list comprehension
    found_helpful, total_helpful = (int(string_split.strip("[],")) for string_split in string_pair.split())
    return(found_helpful, total_helpful)

# define a function to decompose 'helpful' into integers
def decompose_helpful(helpful) : 
    """
    Decompose the 'helpful' column into two values.
    
        Parameters:
            helpful (str): a Series of strings representing pairs of integers, i.e. "[13, 14]"
        
        Returns:
            count_found_helpful (tuple): a tuple of integers representing the number that found the review helpful
            count_total_helpful (tuple): a tuple of integers representing the total number that responded helpful or not
            
        References:
            https://stackoverflow.com/questions/42376201/how-can-i-get-multiple-lists-as-separate-results-from-a-list-comprehension
    """
    # perform a list comprehension on the variable helpful, using decompose_string_pair function
    count_found_helpful, count_total_helpful = zip(*[decompose_string_pair(string_pair) for string_pair in helpful])
    #returns
    return(count_found_helpful, count_total_helpful)


In [74]:
# derive the new integer variables represented in 'helpful'
count_found_helpful, count_total_helpful = decompose_helpful(data_raw['helpful'])

# examine the head of each of the derived variables
print(count_found_helpful[0:5])
print(count_total_helpful[0:5])

(0, 13, 1, 0, 0)
(0, 14, 1, 0, 0)


### 'reviewerName'

In [75]:
# examine the 'reviewerName' variable of the original dataset
print(data_raw['reviewerName'].head())

0    cassandra tu "Yeah, well, that's just like, u...
1                                                Jake
2                       Rick Bennette "Rick Bennette"
3                           RustyBill "Sunday Rocker"
4                                       SEAN MASLANKA
Name: reviewerName, dtype: object


While the reviewer name may be useful in certain contexts, it makes sense to drop this column for the initial approach to modeling, as it does not pertain to the content or helpfulness of the review itself.

### 'asin'

In [77]:
# examine the 'asin' variable of the original dataset
print(data_raw['asin'].head())
print(data_raw['asin'].tail())

0    1384719342
1    1384719342
2    1384719342
3    1384719342
4    1384719342
Name: asin, dtype: object
10256    B00JBIVXGC
10257    B00JBIVXGC
10258    B00JBIVXGC
10259    B00JBIVXGC
10260    B00JBIVXGC
Name: asin, dtype: object


The column 'asin' represents the product ID, as per the data dictionary from Kaggle. This variable may be useful for splitting train-test-validate, but review time has already been selected as the basis for this initial exploration. For a peak into areas that may be worth exploring more, it would be good to examine the distribution of the counts of 'asin'.

### 'reviewerID'

In [78]:
# examine the 'reviewerID' variable of the original dataset
print(data_raw['reviewerID'].head())
print(data_raw['reviewerID'].tail())

0    A2IBPI20UZIR0U
1    A14VAT5EAX3D9S
2    A195EZSQDW3E21
3    A2C00NNG1ZQQG2
4     A94QU4C90B1AX
Name: reviewerID, dtype: object
10256    A14B2YH83ZXMPP
10257     A1RPTVW5VEOSI
10258     AWCJ12KBO5VII
10259    A2Z7S8B5U4PAKJ
10260    A2WA8TDCTGUADI
Name: reviewerID, dtype: object


The column 'reviewerID' is redundant with the column 'reviewerName', which has been deemed irrelevant for this project, and 'reviewerName' would be more useful if included in a model, i.e. if someone has the username "grumpy_purchaser_dislikes_oboes", an NLP model may be able to surmise their bias against oboes.